# LAB1 - Trabajando con estadisticas sobre las opciones del SPX

En este articulo vamos a tratar un uso basico de pandas. Crearemos un dataframe de una serie de archivos separados por comas , y lo guardaremos en pickle para su posterior uso.  
Ademas realizaremos un analisis exploratorio de los datos que nos puede dar una vision preliminar de lo que estamos tratanto.

#### Cargamos todas las librerias necesarias 

In [ ]:
import pandas as pd 
import os
import numpy as np 
import seaborn as sns
import scipy 
import datetime 
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 30, 12 
plt.style.use('ggplot')

### Vamos a crear el dataframe

Vamos a declarar una carpeta, donde tendremos nuestros archivos csv que queramos anyadir a nuestro dataframe para su posterior tratamiento

In [ ]:
mainpath = '../../datasets/lab1/'
files = os.listdir(mainpath)
Symbol = 'SPX'
symboldataname = 'spx_stats.pickle'
Symbol2 ='RUT'
symboldataname2 = 'rut_stats.pickle'

Vamos a cargar todos los archivos que hay dentro del mainpath, y indexarlos por quotedate

In [ ]:
datos={}
contador = 1
df = pd.DataFrame()
data = pd.DataFrame()


for file in range(len(files)):
    usefile = files[file]
    print("Archivo numero :" + str(contador) + " de un total de " + str((len(files))) + " archivos")
    contador = contador +1
    ruta = mainpath + "/" + str(usefile) 
    df = pd.read_csv(ruta,parse_dates=True,index_col=['quotedate'])
    datos[usefile] = df
        
data = pd.concat(datos.values(), keys=datos.keys() , sort='True')

In [ ]:
data.head(5)

Como podemos observar, necesitamos manipular el dataframe para adaptarlo a nuestras necesidades, le quitaremos el multinidice, y seleccionaremos sol las columnas que nos interesen.  
ademas tambien lo ordenaremos

In [ ]:
data = data[['symbol','iv30call','calloi','callvol','iv30put','putoi','putvol','iv30mean','totalvol']]
data = data.droplevel(level=0)
data = data.sort_index(axis=0)


In [ ]:
data.head(5)

Ahora, vamos a crear dos dataframes cada uno con un ticker distinto, y los guardaremos en formato pickle para su posterior uso

In [ ]:
symboldata = data[data['symbol'] == Symbol]
symboldata2 = data[data['symbol'] == Symbol2]
symboldata = symboldata[['iv30call','calloi','callvol','iv30put','putoi','putvol','iv30mean','totalvol']]
symboldata2 = symboldata2[['iv30call','calloi','callvol','iv30put','putoi','putvol','iv30mean','totalvol']]
symboldata = symboldata.sort_index(axis=0)
symboldata2 = symboldata2.sort_index(axis=0)
symboldata.to_pickle('/home/py/datasets/pickle'+ '/' + symboldataname)
symboldata2.to_pickle('/home/py/datasets/pickle' + '/' + symboldataname2 )


In [ ]:
symboldata.head(5)

In [ ]:
symboldata2.head(5)

#### Cargamos el pickle, y comparamos la velocidad de carga en comparacion del read csv 

In [ ]:
df1 = pd.read_pickle('/home/py/datasets/pickle'+ '/' + symboldataname)
df2 = pd.read_pickle('/home/py/datasets/pickle'+ '/' + symboldataname2)

Calculamos la correlacion de una ventana de 5 periodos entre dataframes

In [ ]:

df1.iv30put.rolling(5).corr(df1.iv30call.rolling(5)).plot(color='r',alpha=0.3)
plt.legend(['IV CORR'])
df1.putoi.rolling(5).corr(df1.calloi.rolling(5)).plot(color='b',alpha=0.4)
plt.legend(['IO CORR'])
df1.putvol.rolling(5).corr(df1.callvol.rolling(5)).plot(color='g',alpha=0.5)
plt.legend(['VOL CORR'])


In [ ]:
correlacion = df1.corr(method = 'kendall')
correlacion_mean = correlacion.mean() 

Vamos a visualizar las correlaciones entre las variables

In [ ]:
correlacion.plot.bar()

Con esta forma de visualizar la informacion, queda clara, pero aun podriamos visualizarla mucho mejor mediante un mapa de calor

In [ ]:
mask = np.zeros_like(correlacion, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True


plt.figure(figsize=(12,10),dpi=80)
sns.heatmap(correlacion,mask=mask,
            cmap='RdYlGn',center=0, annot=True)


In [ ]:
correlacionintermarket = pd.DataFrame(index=df1.index)
correlacionintermarket['iv30callRUT'] = df2.iv30call
correlacionintermarket['calloiRUT'] = df2.calloi
correlacionintermarket['callvolRUT'] = df2.callvol
correlacionintermarket['iv30putRUT'] = df2.iv30put
correlacionintermarket['putoiRUT'] = df2.putoi
correlacionintermarket['putvolRUT'] = df2.putvol
correlacionintermarket['ivmean30RUT'] = df2.iv30mean
correlacionintermarket['totalvolRUT'] = df2.totalvol
correlacionintermarket = correlacionintermarket.corr(method = 'kendall')
correlacionintermarketmean = correlacionintermarket.mean()

In [ ]:
mask = np.zeros_like(correlacionintermarket, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True


plt.figure(figsize=(12,10),dpi=80)
sns.heatmap(correlacionintermarket,mask=mask,
            cmap='RdYlGn',center=0, annot=True)
